In [1]:
import json
import boto3
import pandas as pd
import ast

from utils.analysis_utils import process_analysis_in_parallel, process_analysis, format_ligne
from utils.database import get_field_value
from pymongo import MongoClient
import certifi
import importlib

In [17]:
# TODO: Remove

import importlib
import utils.analysis_utils
import utils.database

# Reload the modules
importlib.reload(utils.analysis_utils)
importlib.reload(utils.database)

# Now you can import your functions
from utils.analysis_utils import process_analysis_in_parallel, process_analysis, format_ligne
from utils.database import get_field_value


In [3]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"Prod/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)
mongo_client = MongoClient(MONGO_CONNECTION_STRING,tlsCAFile=certifi.where())

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

# Generating Elementary Subjects

After creating all the extractions, our objective is to generate elementary subjects.

We use the following method:

1. For each extraction, we attempt to classify it among the existing elementary subjects.
2. If no subject matches, we generate a new one.
3. As we progress, we also verify that we are not generating duplicates.

## Process

We classify and generate our elementary subjects progressively:

1. Classify each extraction
2. Generate new subjects as needed
3. Check for duplicates

By the end of this process, after classifying everything, we will have generated all our subjects.

## Database Storage

The elementary subjects are pushed into the `elementary_subjects_dev` database.

In [4]:
BRAND = 'ditp_analysis'

BRAND_DESCR = '''
Feedbacks are from French public services.
'''

In [5]:
# getting feedbacks from mongo

from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
])))

from_mongo.shape

(80537, 85)

In [6]:
# Keeping only feedbacks with extractions and splitted_analysis_v2
subdf = from_mongo[from_mongo['extractions'].notna()]
subdf = subdf[subdf['splitted_analysis_v2'].notna()]

# from_csv = pd.read_csv('ditp_test_2.csv')
# subdf = from_csv

subdf['brand_context'] = subdf.apply(format_ligne, axis=1)
subdf = subdf[['_id', 'extractions', 'brand_context']]

print(f"Subdf contains {subdf.shape[0]} rows.")

Subdf contains 148 rows.


### For one feedback

In [15]:
row = subdf.sample().iloc[0]
row.brand_context.split("Full feedback**:")[1]

' Histovec encore en panne\nLe service histovec, utile pour les automobiliste et payé par nos impôts est  encore en panne !\r\n\r\nPourquoi une telle incompétence ?'

In [18]:



res = process_analysis(
    feedback_id=row['_id'],
    extractions=row["extractions"],
    brand_context=row["brand_context"],
    model="gpt-4o-mini",
    brand_name=BRAND,
    language="french",
    extractions_column="extractions",
    should_update_mongo=True
)

res


{'topics': ['Fonctionnement du Site : Service en Panne'], 'justification': "Le feedback indique que le service Histovec est en panne, ce qui reflète un problème de fonctionnement du site. Aucune des catégories existantes ne correspond spécifiquement à un problème de service en panne, ce qui justifie la création d'une nouvelle catégorie.", 'is_new_topic': True}
{'topics': ['Fonctionnement du Site : Service en Panne'], 'justification': "Le feedback indique clairement que le service Histovec est en panne, ce qui correspond directement à la catégorie 'Fonctionnement du Site : Service en Panne'.", 'is_new_topic': False}
{'topics': ['Fonctionnement du Site : Service en Panne'], 'justification': "Le feedback exprime une frustration face à l'indisponibilité du service Histovec, ce qui correspond à la catégorie 'Fonctionnement du Site : Service en Panne'.", 'is_new_topic': False}
{'topics': ['Fonctionnement du Site : Service en Panne'], 'justification': "Le feedback indique clairement que le se

{'id': 'ditp_analysis/4620419',
 'extractions': [{'sentiment': 'NEGATIVE',
   'extraction': 'Histovec en panne',
   'text': 'Histovec encore en panne',
   'elementary_subjects': ['Fonctionnement du Site : Service en Panne'],
   'topics': []},
  {'sentiment': 'NEGATIVE',
   'extraction': 'Histovec en panne',
   'text': 'utile pour les automobiliste et payé par nos impôts est  encore en panne',
   'elementary_subjects': ['Fonctionnement du Site : Service en Panne'],
   'topics': []},
  {'sentiment': 'NEGATIVE',
   'extraction': 'Incompétence du service Histovec',
   'text': 'Pourquoi une telle incompétence',
   'elementary_subjects': ['Fonctionnement du Site : Service en Panne'],
   'topics': []},
  {'sentiment': 'NEGATIVE',
   'extraction': 'Service utile pour les automobilistes mais en panne',
   'text': 'Le service histovec',
   'elementary_subjects': ['Fonctionnement du Site : Service en Panne'],
   'topics': []},
  {'sentiment': 'NEGATIVE',
   'extraction': 'Service utile pour les a

### For multiple feedbacks

In [180]:

importlib.reload(utils.analysis_utils)
from utils.analysis_utils import run_analysis_full_parallel

In [187]:
feedbacks_sample = subdf.sample(5)

In [188]:

res = process_analysis_in_parallel(
    feedbacks=feedbacks_sample,
    brand_name=BRAND,
    language='french',
    model="gpt-4o-mini",
    save_to_mongo=True
)

# res

Processing chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Updating feedback ditp_analysis/4306751 with updates: {'extractions': [], 'splitted_analysis_v2': [], 'topics_v2': []}
Updating feedback ditp_analysis/3971879 with updates: {'extractions': [], 'splitted_analysis_v2': [], 'topics_v2': []}
Updating feedback ditp_analysis/389783 with updates: {'extractions': [{'sentiment': 'NEGATIVE', 'extraction': "Récupération de la carte d'identité de l'enfant", 'text': "Pour récupérer la carte d'identité de mon enfant", 'elementary_subjects': ['Récupération de Documents : Attente et Exigences Administratives'], 'topics': []}, {'sentiment': 'NEGATIVE', 'extraction': 'Rendez-vous multiples en même temps', 'text': 'plusieurs personnes ont rendez vous en même temps', 'elementary_subjects': ['Récupération de Documents : Attente et Exigences Administratives'], 'topics': []}, {'sentiment': 'NEGATIVE', 'extraction': 'Attente et frustration générées', 'text': 'Ce qui génère attente et frustration', 'elementary_subjects': ['Récupération de Documents : Attente e

Processing chunks: 100%|██████████| 1/1 [00:30<00:00, 31.00s/it]

[check_duplicates()] invalid syntax (<string>, line 0)


## Check elementary subjects that we generated

In [8]:
from utils.database import get_elementary_subjects

In [9]:
negative_elementary_subjects = get_elementary_subjects(BRAND, 'negative')

print(len(negative_elementary_subjects))

[x['elementary_subject'] for x in negative_elementary_subjects[:5]]

3


['Assistance : Manque de Réponses et de Support',
 'Récupération de Documents : Attente et Exigences Administratives',
 'Système de Notation : Inexactitude des Retours Utilisateurs']

In [13]:
# row = subdf.sample().iloc[0]
# extractions = get_field_value(
#     feedback_id=row['_id'],
#     field_name='extractions',
# )
# res = classify_one_feedback(
#     feedback_id=row['_id'],
#     extractions=extractions,
#     model="gpt-4o-mini",
#     brand=BRAND,
#     brand_descr=BRAND_DESCR,
#     language="french",
#     extractions_column="extractions",
#     update_mongo=False
# )

# # res

Sentence : J’ai fait ma demande d’Aspa en juillet 2021.
Extraction : Demande d'Aspa en attente depuis juillet 2021
Elementary Subjects : ["Service : Délais d'attente pour les demandes : Demande d'Aspa en attente"]
Sentence : Mon dossier est complet et il est toujours en attente depuis !
Extraction : Demande d'Aspa en attente depuis juillet 2021
Elementary Subjects : ["Service : Délais de traitement des demandes : Demande d'Aspa en attente"]
Sentence : je suis passée par la CARSAT DE L’ORNE ou j’ai trouvé une super écoute !
Extraction : Bonne écoute de la CARSAT DE L’ORNE
Elementary Subjects : ["Service : Qualité de l'écoute des agents"]
Sentence : J’ai reçu un mail sur mon espace client me disant que mon dossier serait traité sous 7 jours ouvrés,
Extraction : Dossier non traité dans les délais annoncés
Elementary Subjects : ['Service : Délai de traitement : Dossier non traité dans les délais annoncés']
Sentence : et bien non !
Extraction : Dossier non traité dans les délais annoncés
El